In [2]:
import pandas as pd
import numpy as np
import time
from scipy.spatial.distance import pdist

embedding_cols = [
#     'year',
    'acousticness',
    'danceability',
    'duration_ms',
    'energy',
    'instrumentalness',
    'key',
    'liveness',
    'loudness',
    'mode',
    'speechiness',
    'tempo',
    'time_signature',
    'valence'
]

msd = pd.read_hdf('data/full_msd_with_audio_features.h5', key='df')
msd = msd[['song_id'] + embedding_cols]
msd.head()

def get_list_dissimilarity(song_ids):
    song_vectors = msd.loc[msd['song_id'].isin(song_ids)][embedding_cols].values
    return np.mean(pdist(song_vectors, 'cosine'))

feature_MUSIC_dict = {
    'danceability': np.array([-0.37, 0.05, -0.35, 0.08, 0.43]),
    'energy': np.array([-0.64, -0.46, -0.13, 0.66, -0.03]),
    'instrumentalness': np.array([0.20, -0.47, 0.28, 0.09, -0.01]),
    'liveness': np.array([-0.69, -0.12, -0.07, 0.43, 0.02]),
    'loudness': np.array([-0.58, -0.19, -0.44, 0.79, -0.21]),
    'valence': np.array([-0.04, 0.18, 0.24, -0.34, 0.18]),
}
feature_MUSIC_matrix = [MUSIC for MUSIC in feature_MUSIC_dict.values()]

def get_MUSIC(song_ids):
    song_vectors = msd.loc[msd['song_id'].isin(song_ids)][list(feature_MUSIC_dict.keys())].values
    return np.sum(np.dot(song_vectors, feature_MUSIC_matrix), axis=0) / len(song_ids)

# song_ids = list(msd.head(1000).song_id)
# start = time.time()
# print(f'Dissim was {get_list_dissimilarity(song_ids)}')
# print(f'It took {time.time() - start}s')

In [33]:
triplets = pd.read_hdf('data/triplets.h5')[['song_id', 'user_id']]
triplets.head()

,song_id,user_id
0,SOAKIMP12A8C130995,b80344d063b5ccb3212f76538f3d9e43d87dca9e
1,SOAPDEY12A81C210A9,b80344d063b5ccb3212f76538f3d9e43d87dca9e
2,SOBBMDR12A8C13253B,b80344d063b5ccb3212f76538f3d9e43d87dca9e
3,SOBFNSP12AF72A0E22,b80344d063b5ccb3212f76538f3d9e43d87dca9e
4,SOBFOVM12A58A7D494,b80344d063b5ccb3212f76538f3d9e43d87dca9e


In [34]:
gb = triplets.groupby('user_id')
# triplets.head(120)['user_id'].nunique()
triplets = triplets.head(120)
def yolo(vals):
#     print(vals)
    return len(vals)
def polo(vals):
#     print(vals)
    return len(vals)

df = gb.agg([yolo, polo])
df.columns = df.columns.droplevel(0)
df = df.reset_index()
# df.columns
df.head()

In [60]:
gb = triplets.groupby('user_id')
# triplets.head(120)['user_id'].nunique()
triplets = triplets.head(120)
def yolo(vals):
#     print(vals)
    return len(vals)
def polo(vals):
#     print(vals)
    return len(vals)

df = gb.agg([yolo, polo])
df.columns = df.columns.droplevel(0)
df = df.reset_index()
# df.columns
df.head()

,user_id,yolo,polo
0,85c1f87fea955d09b4bec2e36aee110927aedf9a,16,16
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,104,104


In [38]:
gb.get_group('85c1f87fea955d09b4bec2e36aee110927aedf9a')

,song_id,user_id
104,SOACWYB12AF729E581,85c1f87fea955d09b4bec2e36aee110927aedf9a
105,SOAUSXX12A8C136188,85c1f87fea955d09b4bec2e36aee110927aedf9a
106,SOBVAHM12A8C13C4CB,85c1f87fea955d09b4bec2e36aee110927aedf9a
107,SODJTHN12AF72A8FCD,85c1f87fea955d09b4bec2e36aee110927aedf9a
108,SOELXOV12A8C13616B,85c1f87fea955d09b4bec2e36aee110927aedf9a
109,SOGGLLR12A8C13C471,85c1f87fea955d09b4bec2e36aee110927aedf9a
110,SOHANDU12A8C13C47F,85c1f87fea955d09b4bec2e36aee110927aedf9a
111,SOIDFHN12A8C13ABAC,85c1f87fea955d09b4bec2e36aee110927aedf9a
112,SOJONCB12A8C13B8F5,85c1f87fea955d09b4bec2e36aee110927aedf9a
113,SOJQTIA12AF729F6BF,85c1f87fea955d09b4bec2e36aee110927aedf9a
